In [ ]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
import random 
import numpy as np
import warnings
import tensorflow as tf 
from preprocess_days_stats import preprocess_cumulative_stats, preprocess_match_days
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score
from preprocess_features import preprocess_columns, preprocess_columns_with_odds
from tensorflow.keras import layers
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams, plot_loss_curve
from imblearn.over_sampling import RandomOverSampler

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")


df_giornate = preprocess_match_days(r"c:\Users\Hp\Documents\Serie_A_dump\csv_serie_a")
df_giornate

# Capiamo l'accuracy da battere

In [ ]:
conditions_1 = [
      ( (df_giornate[f'home_win_odds'] <= df_giornate['draw_odds']) & (df_giornate[f'home_win_odds'] <= df_giornate[f'away_win_odds'] )), 
      ( (df_giornate['draw_odds'] <= df_giornate[f'away_win_odds']) & (df_giornate['draw_odds'] <= df_giornate[f'home_win_odds'] )), 
      ( (df_giornate[f'away_win_odds'] <= df_giornate['draw_odds']) & (df_giornate[f'away_win_odds'] <= df_giornate[f'home_win_odds'] ))
      ]
values=['H','D','A']

predizioni_bookmakers =np.select(conditions_1, values)
risultati_effettivi = np.array(df_giornate['ft_result'])
round(accuracy_score(predizioni_bookmakers, risultati_effettivi),3)

l'accuracy da battere è 54.5%

# Salviamo i risultati della baseline in un csv

In [ ]:
import csv

# Nome del file CSV
nome_file = "results/baseline_KNN.csv"

# Apri il file CSV in modalità scrittura
with open(nome_file, mode='w', newline='') as file_csv:
    # Creazione dell'oggetto writer
    writer = csv.writer(file_csv)

    # Scrivi l'intestazione nel file CSV
    writer.writerow(['giorni_cumulativi', 'vanumero_colonne', 'oversample', 'vicini', 'val_accuracy', 'train_accuracy'])

# KNN baseline

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score
from preprocess_features import preprocess_columns

for giorni_cumulativi in range(10):
    giorni_cumulativi = giorni_cumulativi+1
    df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)
    for numero_colonne in ['all','less','few']:    
        for oversample in [True, False]:
            (X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
               X_train_df, X_valid_df, X_test_df, Train_labels, Valid_labels, Test_labels) = preprocess_columns(
                df_Serie_A_days,numero_colonne,giorni_cumulativi, oversample=oversample)

            best_accuracy=0
            for vicini in range(3,40): #prendiamo diversi numeri di vicini possibili
                knn_model = KNeighborsClassifier(n_neighbors=3*vicini) #addestriamo il modello con questo numero di vicini 
                knn_model.fit(X_train_norm, Train_labels_encoded) 
                y_valid = knn_model.predict(X_valid_norm)
                val_accuracy = round(accuracy_score(Valid_labels_encoded, y_valid),2)
                y_train = knn_model.predict(X_train_norm)
                train_accuracy = round(accuracy_score(Train_labels_encoded, y_train),2)

                if val_accuracy > best_accuracy:
                    best_accuracy = val_accuracy
                    
                    # Scrivi i dati nel file CSV
                    with open(nome_file, mode='a', newline='') as file_csv:
                        writer = csv.writer(file_csv)
                        writer.writerow([giorni_cumulativi, numero_colonne, oversample, vicini*3, val_accuracy, train_accuracy])            

In [ ]:
baseline_KNN_df = pd.read_csv(r'results/baseline_KNN.csv').sort_values(['oversample','val_accuracy'],ascending=[False,False])
baseline_KNN_df.head(8)

In [ ]:
baseline_KNN_df = pd.read_csv(r'results/baseline_KNN.csv').sort_values(['oversample','val_accuracy'],ascending=[True,False])
baseline_KNN_df.head(8)

# Dense neural network

In [ ]:
import tensorflow as tf 

# Create a function to implement a ModelCheckpoint callback with a specific filename 
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=0, # only output a limited amount of text
                                            monitor='val_loss',
                                            save_best_only=True) # save only the best model to file

## Iniziamo con poche features e 3 giorni cumulativi

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

In [ ]:
def create_and_compile_dense(num_dense_layers, first_dropout, other_dropouts, first_num_neurons,
                              other_num_neurons, input_shape, first_activation='relu', other_activations='relu'):

    inputs = layers.Input(shape=(input_shape,))
    x = layers.Dense(first_num_neurons, activation=first_activation)(inputs)
    x = layers.Dropout(first_dropout)(x)  # Aggiunto il layer di dropout per ridurre overfitting
    for i in range(num_dense_layers-1):
        x = layers.Dense(other_num_neurons, activation=other_activations)(x)
        x = layers.Dropout(other_dropouts)(x) 
    outputs = layers.Dense(3, activation='softmax')(x)
    dense_model = tf.keras.Model(inputs, outputs, name='model_1_dense')

    # compilo il modello 
    dense_model.compile(
        loss='sparse_categorical_crossentropy',
        metrics = 'accuracy',
        optimizer = tf.keras.optimizers.Adam()
    )

    return dense_model

In [ ]:
for giorni_cumulativi in [5]:
    # calcolo il dataframe con i giorni cumulativi 
    df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)
    print(f'giorni cumulativi: {giorni_cumulativi}')
    # seleziono il numero di features da usare nel modello 
    for numero_colonne in ['all','less','few']:
        print(f'numero colonne: {numero_colonne}')    
        X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded = preprocess_columns(
            df_Serie_A_days,numero_colonne,giorni_cumulativi)
        input_shape = X_train_norm.shape[1]
        # fast preprocessing 
        X_train_norm = tf.data.Dataset.from_tensor_slices((X_train_norm, Train_labels_encoded))
        X_valid_norm = tf.data.Dataset.from_tensor_slices((X_valid_norm, Valid_labels_encoded))
        X_train_norm = X_train_norm.batch(32).prefetch(tf.data.AUTOTUNE) #Autotune è per dirgli di prefetchare tanti dati quanti può
        X_valid_norm = X_valid_norm.batch(32).prefetch(tf.data.AUTOTUNE)
        #iniziamo a creare i modelli 
        for num_dense_layers in [2,3,4]:
            for first_dropout in [0.0, 0.2, 0.4, 0.6]:
                for other_dropouts in [0.0, 0.2, 0.4, 0.6]:
                    for first_num_neurons in [32,64,128]:
                        for other_num_neurons in [8, 16, 32]:
                            other_activations = 'relu'
                            first_activation = 'relu'
                            model_1 = create_and_compile_dense(num_dense_layers, first_dropout, other_dropouts, first_num_neurons,
                                    other_num_neurons, input_shape, first_activation, other_activations)

                                # fitto il modello 
                            model_1.fit(
                                    x = X_train_norm,
                                    epochs = 15,
                                    verbose=0,
                                    validation_data = X_valid_norm,
                                    callbacks = [
                                        CSVLoggerCallbackParams('results.csv','model_1_dense', num_dense_layers, first_dropout, other_dropouts, first_num_neurons,
                                                                    other_num_neurons, first_activation, other_activations), 
                                        #create_model_checkpoint(model_name= 'Dense_model_best')
                                                ] 
                                )

In [ ]:
df_results_5_days = pd.read_csv(r'C:\Users\Hp\Serie_A\results_5_days.csv', header=None)
df_results_1_2_4 = pd.read_csv(r'C:\Users\Hp\Serie_A\results_1_2_4.csv', header=None)
df_results =pd.concat([df_results_5_days,df_results_1_2_4],ignore_index=True)
df_results.columns = ['experiment', 'num_dense_layers', 'first_dropout', 'other_dropouts', 'first_num_neurons',
                                    'other_num_neurons', 'first_activation', 'other_activations', 'epoch',
                                    'loss', 'accuracy','val_loss','val_accuracy']
df_results.sort_values(['val_loss'], inplace=True)
df_results.head(3)

### il modello denso migliore 

giorni =5, colonne = less, val_loss: 0.9921 - val_accuracy: 0.5247

giorni = 2, colonne = few, val_loss: 0.9895 - val_accuracy: 0.5220

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

giorni_cumulativi = 2
# calcolo il dataframe con i giorni cumulativi 
df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)

#scelgo le colonne che voglio usare
numero_colonne = 'less'
print(f'numero colonne: {numero_colonne}')    
(X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
               X_train_df, X_valid_df, X_test_df, Train_labels, Valid_labels, Test_labels) = preprocess_columns(
                                                                                        df_Serie_A_days,numero_colonne,giorni_cumulativi, oversample=False)
input_shape = X_train_norm.shape[1]
Dataset_train_norm = tf.data.Dataset.from_tensor_slices((X_train_norm))
Train_labels_encoded = tf.data.Dataset.from_tensor_slices(Train_labels_encoded) # make labels
Dataset_train_norm = tf.data.Dataset.zip((Dataset_train_norm, Train_labels_encoded))

Dataset_valid_norm = tf.data.Dataset.from_tensor_slices((X_valid_norm))
Valid_labels_encoded = tf.data.Dataset.from_tensor_slices(Valid_labels_encoded) # make labels
Dataset_valid_norm = tf.data.Dataset.zip((Dataset_valid_norm, Valid_labels_encoded))

Dataset_test_norm = tf.data.Dataset.from_tensor_slices((X_test_norm))
Test_labels_encoded = tf.data.Dataset.from_tensor_slices(Test_labels_encoded) # make labels
Dataset_test_norm = tf.data.Dataset.zip((Dataset_test_norm, Test_labels_encoded))

Dataset_train_norm = Dataset_train_norm.batch(32).prefetch(tf.data.AUTOTUNE) #Autotune è per dirgli di prefetchare tanti dati quanti può
Dataset_valid_norm = Dataset_valid_norm.batch(32).prefetch(tf.data.AUTOTUNE)
Dataset_test_norm = Dataset_test_norm.batch(32).prefetch(tf.data.AUTOTUNE)

#creo e compilo il modello 
model_1 = create_and_compile_dense(num_dense_layers=3, first_dropout=0.2, other_dropouts=0.1, first_num_neurons=32,
                            other_num_neurons=8, input_shape=input_shape, first_activation='relu', other_activations='relu')

# fitto il modello 
history_1 = model_1.fit(
                x = Dataset_train_norm,
                epochs = 15,
                verbose = 1,
                batch_size = 128,
                validation_data = Dataset_valid_norm,
                callbacks = [
                                #CSVLoggerCallbackParams('results.csv','model_1_dense', num_dense_layers, first_dropout, other_dropouts, first_num_neurons,
                                #                other_num_neurons, first_activation, other_activations), 
                                #create_model_checkpoint(model_name= 'Dense_model_best') 
                            ] 
                )

### VAlutiamo il modello

In [ ]:
model_1 = tf.keras.models.load_model(r'C:\Users\Hp\Serie_A\model_experiments\Dense_model_best')
model_1.evaluate(Dataset_valid_norm)

In [ ]:
plot_loss_curve(history_1)

Si nota un po' di overfitting nel nostro modelo

In [ ]:
result_df = pd.merge(X_valid_df, df_giornate, how='inner', on=['stagione', 'awayteam','hometeam']).reset_index()
len(result_df), len(result_df['stagione'])

In [ ]:
# Visualizziamo un po' di risultati 
model_1_pred_probs = model_1.predict(X_valid_norm)
model_1_prob = model_1.predict(X_valid_norm).max(axis=1)
model_1_predictions = model_1_pred_probs.argmax(axis=1)
model_1_compare = pd.DataFrame({
                                'stagione': list( result_df['stagione'] ),
                                'hometeam': list( result_df['hometeam'] ),
                                'awayteam': list( result_df['awayteam'] ),
                                'preds': model_1_predictions, 
                                'result': Valid_labels, 
                                'pred_prob': model_1_prob,
                                'home_win_odds': list( result_df['home_win_odds'] ),
                                'draw_odds': list( result_df['draw_odds'] ),
                                'away_win_odds': list( result_df['away_win_odds'])
                                })
model_1_compare['bookmakers_pred'] = np.argmin(result_df[['home_win_odds', 'draw_odds', 'away_win_odds']].fillna(0.0).to_numpy(), axis=1)
model_1_compare['bookmakers_prob'] = np.nanmin(result_df[['home_win_odds', 'draw_odds', 'away_win_odds']].fillna(0.0).to_numpy(), axis=1)

# Assegno ai valori encoded dei valori più comprensibili per vittoria pareggio sconfitta
conditions = [
(model_1_compare['preds'] == 2),  # Condizione per Home Win
(model_1_compare['preds'] == 1),  # Condizione per Away Win
(model_1_compare['preds'] == 0)   # Condizione per Draw
]
conditions_bookmakers = [
(model_1_compare['bookmakers_pred'] == 0),  # Condizione per Home Win
(model_1_compare['bookmakers_pred'] == 2),  # Condizione per Away Win
(model_1_compare['bookmakers_pred'] == 1)   # Condizione per Draw
]

# Valori corrispondenti alle condizioni
values = ['W', 'L', 'D']

# Creazione della nuova colonna 'result' e 'points
model_1_compare['preds'] = np.select(conditions, values)
model_1_compare['bookmakers_pred'] = np.select(conditions_bookmakers, values)
model_1_compare['is_correct'] = model_1_compare['preds'] == model_1_compare['result']

len(model_1_compare)

In [ ]:
# Visualizziamo i risultati più regenti 
model_1_compare.sort_values(['stagione'], ascending = False).head(10)

In [ ]:
# Visualizziamo i nostri riusltati corretti e quelli sbagliati della bookmakers 
model_1_compare[(model_1_compare['is_correct']) & (model_1_compare['result']!=model_1_compare['bookmakers_pred'])].head(10)

In [ ]:
# visualizziamo i risultati più sbagliati
model_1_compare[(~model_1_compare['is_correct'])].sort_values(['pred_prob'], ascending=False).head(10)

Si vede che molti errori sono dati da squadre blasonate che giocano in casa

# CONV1D model

In [ ]:
def create_and_compile_conv_1d(num_conv_layers, first_dropout, other_dropouts, first_filters_num, first_kernel_size,
                               other_num_filters, other_kernel_sizem, input_shape, dense_neurons, first_activation='relu', other_activations='relu'):
    inputs = layers.Input(shape=(input_shape,))
    x = layers.Reshape((input_shape, 1))(inputs) # add an extra dimension for timesteps
    x = layers.Conv1D(filters=first_filters_num, kernel_size=first_kernel_size, activation=first_activation)(x)
    x = layers.Dropout(first_dropout)(x) 
    for i in range(num_conv_layers-1):
        x = layers.Conv1D(filters=other_num_filters, kernel_size=other_kernel_sizem, activation=first_activation)(x)
        x = layers.Dropout(other_dropouts)(x) 
    x = layers.Flatten()(x)
    x = layers.Dense(dense_neurons)(x)
    x = layers.Dropout(other_dropouts)(x) 
    outputs = layers.Dense(3, activation='softmax')(x)
    conv_model = tf.keras.Model(inputs, outputs, name='model_1_dense')

    # compilo il modello 
    conv_model.compile(
        loss='sparse_categorical_crossentropy',
        metrics = 'accuracy',
        optimizer = tf.keras.optimizers.Adam()
    )

    return conv_model

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

giorni_cumulativi = 3
# calcolo il dataframe con i giorni cumulativi 
df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)

#scelgo le colonne che voglio usare
numero_colonne = 'less'
print(f'numero colonne: {numero_colonne}')    
(X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
               X_train_df, X_valid_df, X_test_df, Train_labels, Valid_labels, Test_labels) = preprocess_columns(
                                                                                        df_Serie_A_days,numero_colonne,giorni_cumulativi, oversample=False)
input_shape = X_train_norm.shape[1]

# fast preprocessing 
Dataset_train_norm = tf.data.Dataset.from_tensor_slices((X_train_norm, Train_labels_encoded))
Dataset_valid_norm = tf.data.Dataset.from_tensor_slices((X_valid_norm, Valid_labels_encoded))
Dataset_train_norm = Dataset_train_norm.batch(32).prefetch(tf.data.AUTOTUNE) #Autotune è per dirgli di prefetchare tanti dati quanti può
Dataset_valid_norm = Dataset_valid_norm.batch(32).prefetch(tf.data.AUTOTUNE)

#creo e compilo il modello 
model_2 = create_and_compile_conv_1d(num_dense_layers=3, first_dropout=0.2, other_dropouts=0.0,
                            other_num_neurons=8, input_shape=input_shape, first_activation='relu', other_activations='relu')

# fitto il modello 
history_2 = model_2.fit(
                x = Dataset_train_norm,
                epochs = 15,
                verbose = 1,
                validation_data = Dataset_valid_norm,
                callbacks = [
                                #CSVLoggerCallbackParams('results.csv','model_1_dense', num_dense_layers, first_dropout, other_dropouts, first_num_neurons,
                                #                other_num_neurons, first_activation, other_activations), 
                                #create_model_checkpoint(model_name= 'CONV1D_model_best') 
                            ] 
                )

# Unisco il modello dense che ho creato sopra con uno che tenga conto delle quote 

In [ ]:
# Create a function to implement a ModelCheckpoint callback with a specific filename 
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=0, # only output a limited amount of text
                                            monitor='val_loss',
                                            save_best_only=True) # save only the best model to file

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

giorni_cumulativi = 2
# calcolo il dataframe con i giorni cumulativi 
df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)

#scelgo le colonne che voglio usare
numero_colonne = 'less'
print(f'numero colonne: {numero_colonne}')    
(X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
    X_train_df, X_valid_df, X_test_df, Train_labels, Valid_labels, Test_labels,Train_odds_df,Valid_odds_df,Test_odds_df) = preprocess_columns_with_odds(
                                                                                        df_Serie_A_days,numero_colonne,giorni_cumulativi, oversample=False)
input_shape = X_train_norm.shape[1]

print(X_valid_norm.shape,X_test_norm.shape,len(Test_odds_df))

# fast preprocessing 
Dataset_train_norm = tf.data.Dataset.from_tensor_slices((X_train_norm, Train_odds_df))
Train_labels_encoded = tf.data.Dataset.from_tensor_slices(Train_labels_encoded) # make labels
Dataset_train_norm = tf.data.Dataset.zip((Dataset_train_norm, Train_labels_encoded))

Dataset_valid_norm = tf.data.Dataset.from_tensor_slices((X_valid_norm, Valid_odds_df))
Valid_labels_encoded = tf.data.Dataset.from_tensor_slices(Valid_labels_encoded) # make labels
Dataset_valid_norm = tf.data.Dataset.zip((Dataset_valid_norm, Valid_labels_encoded))

Dataset_Test_norm = tf.data.Dataset.from_tensor_slices((X_test_norm, Test_odds_df))
Test_labels_encoded = tf.data.Dataset.from_tensor_slices(Test_labels_encoded) # make labels
Dataset_Test_norm = tf.data.Dataset.zip((Dataset_Test_norm, Test_labels_encoded))

Dataset_train_norm = Dataset_train_norm.batch(32).prefetch(tf.data.AUTOTUNE) #Autotune è per dirgli di prefetchare tanti dati quanti può
Dataset_valid_norm = Dataset_valid_norm.batch(32).prefetch(tf.data.AUTOTUNE)
Dataset_Test_norm = Dataset_Test_norm.batch(32).prefetch(tf.data.AUTOTUNE)

print(Dataset_valid_norm)
Dataset_Test_norm

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

#creo e compilo il modello denso
inputs = layers.Input(shape=(input_shape,))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dropout(0.2)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(16, activation='relu')(x)
x = layers.Dropout(0.1)(x) 
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dropout(0.1)(x) 
dense_model = tf.keras.Model(inputs, outputs, name='model_3_dense')

#creo un modello che interpreti le odds
inputs_odds = layers.Input(shape=(3,))
x = layers.Dense(8, activation='relu')(inputs_odds)
outputs_odds = layers.Dense(3, activation='linear')(x)
model_3_odds = tf.keras.Model(inputs_odds, outputs_odds, name='model_3_odds')

#unisco i due modelli 
# 3. Concatenate token and char inputs 
odds_concat = layers.Concatenate(name="odds_dense_model")([model_3_odds.output,
                                                                  dense_model.output])
x = layers.Dense(16, activation='tanh')(odds_concat)
x =layers.Dropout(0.5)(x)
output_layer = layers.Dense(3, activation = 'softmax')(x)

#Costruisco il modello con char e token input
model_3 =tf.keras.Model(
    inputs=[dense_model.input, model_3_odds.input],
    outputs=output_layer,
    name='model_4_odds_concat'
)

# compilo il modello 
model_3.compile(
    loss='sparse_categorical_crossentropy',
    metrics = 'accuracy',
    optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)
)

#Se vuoi ridurre il tasso di apprendimento del 10% ogni 10 epoche
def scheduler(epoch, lr):
    if epoch == 5:
        return lr * 0.01  # Riduci il tasso di apprendimento del 10% ogni 10 epoche
    else:
        return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

# fitto il modello 
history_3 = model_3.fit(
                x = Dataset_train_norm,
                epochs = 6,
                verbose = 1,
                batch_size = 32,
                validation_data = Dataset_valid_norm,
                callbacks = [
                                CSVLoggerCallbackParams('results.csv','model_3_dense', giorni_cumulativi,numero_colonne, 3, 0.2, 0.0, 32,
                                                8, 'relu', 'relu'), 
                                lr_scheduler,
                                create_model_checkpoint(model_name= 'Dense_model_best_1') 
                            ] 
                )

## Valuto il modello migliore

In [ ]:
model_3 = tf.keras.models.load_model(r'C:\Users\Hp\Serie_A\model_experiments\Dense_model_best_1')
model_3.evaluate(Dataset_Test_norm)

In [ ]:
plot_loss_curve(history_3)

### Vediamo come performano i nostri risultati rispetto a quelli della bookmakers 

In [ ]:
result_df = pd.merge(X_valid_df, df_giornate, how='inner', on=['stagione', 'awayteam','hometeam']).reset_index()

# Visualizziamo un po' di risultati 
model_3_pred_probs = model_3.predict((X_valid_norm, Valid_odds_df))
model_3_prob = model_3_pred_probs.max(axis=1)
model_3_predictions = model_3_pred_probs.argmax(axis=1)
model_3_compare = pd.DataFrame({
                                'stagione': list( result_df['stagione'] ),
                                'hometeam': list( result_df['hometeam'] ),
                                'awayteam': list( result_df['awayteam'] ),
                                'preds': model_3_predictions, 
                                'result': Valid_labels, 
                                'pred_prob': model_3_prob,
                                'home_win_odds': list( result_df['home_win_odds'] ),
                                'draw_odds': list( result_df['draw_odds'] ),
                                'away_win_odds': list( result_df['away_win_odds'])
                                })
model_3_compare['bookmakers_pred'] = np.argmin(result_df[['home_win_odds', 'draw_odds', 'away_win_odds']].fillna(0.0).to_numpy(), axis=1)
model_3_compare['bookmakers_prob'] = np.nanmin(result_df[['home_win_odds', 'draw_odds', 'away_win_odds']].fillna(0.0).to_numpy(), axis=1)

# Assegno ai valori encoded dei valori più comprensibili per vittoria pareggio sconfitta
conditions = [
(model_3_compare['preds'] == 2),  # Condizione per Home Win
(model_3_compare['preds'] == 1),  # Condizione per Away Win
(model_3_compare['preds'] == 0)   # Condizione per Draw
]
conditions_bookmakers = [
(model_3_compare['bookmakers_pred'] == 0),  # Condizione per Home Win
(model_3_compare['bookmakers_pred'] == 2),  # Condizione per Away Win
(model_3_compare['bookmakers_pred'] == 1)   # Condizione per Draw
]

# Valori corrispondenti alle condizioni
values = ['W', 'L', 'D']

# Creazione della nuova colonna 'result' e 'points
model_3_compare['preds'] = np.select(conditions, values)
model_3_compare['bookmakers_pred'] = np.select(conditions_bookmakers, values)
model_3_compare['is_correct'] = model_3_compare['preds'] == model_3_compare['result']
model_3_compare['money_won'] = model_3_compare['pred_prob']*model_3_compare['bookmakers_prob']

len(model_3_compare)

In [ ]:
# Visualizziamo i nostri riusltati corretti e quelli sbagliati della bookmakers
test = model_3_compare[(model_3_compare['is_correct']) & (model_3_compare['result']!=model_3_compare['bookmakers_pred'])].sort_values(['money_won'], ascending=False).head(100)
print(f'il nostro modello ha performato meglio in {len(test)} su {len(model_3_compare)}') 
test.head()


### IL modello sembra completamente incapace di prevedere i pareggi

In [ ]:
len(model_3_compare[(model_3_compare['result']=='D')]), len(model_3_compare[(model_3_compare['result']=='D')&(model_3_compare['is_correct'])])

### Vincita soldi

In [ ]:
# vediamo in quanti casi avremmo vinto soldi 
test_1 = model_3_compare[(model_3_compare['is_correct']) & (model_3_compare['money_won']>1)].sort_values(['money_won'], ascending=False)
correct_results = model_3_compare[(model_3_compare['is_correct'])]
print(f'il nostro modello ha performato meglio in {len(test_1)} su {len(model_3_compare)} totali e {len(correct_results)} casi in cui la predizione è corretta ') 
test_1.head(20)

In [ ]:
# vediamo in quanti casi avremmo perso soldi 
test_2 = model_3_compare[(model_3_compare['money_won']>1)]
wrong_results = model_3_compare[(~model_3_compare['is_correct'])].sort_values(['money_won'], ascending=False)
test_2.head(20)

In [ ]:
# vediamo quanti soldi avrei vinto e quanti perso
test_1['bookmakers_prob'].sum()-len(test_2)

## Facciamo gli stessi test sul dataset di test

In [ ]:
result_df = pd.merge(X_test_df, df_giornate, how='inner', on=['stagione', 'awayteam','hometeam']).reset_index()

# Visualizziamo un po' di risultati 
model_3_pred_probs = model_3.predict((X_test_norm, Test_odds_df))
model_3_prob = model_3_pred_probs.max(axis=1)
model_3_predictions = model_3_pred_probs.argmax(axis=1)
model_3_compare = pd.DataFrame({
                                'stagione': list( result_df['stagione'] ),
                                'hometeam': list( result_df['hometeam'] ),
                                'awayteam': list( result_df['awayteam'] ),
                                'preds': model_3_predictions, 
                                'result': Test_labels, 
                                'pred_prob': model_3_prob,
                                'home_win_odds': list( result_df['home_win_odds'] ),
                                'draw_odds': list( result_df['draw_odds'] ),
                                'away_win_odds': list( result_df['away_win_odds'])
                                })
model_3_compare['bookmakers_pred'] = np.argmin(result_df[['home_win_odds', 'draw_odds', 'away_win_odds']].fillna(0.0).to_numpy(), axis=1)
model_3_compare['bookmakers_prob'] = np.nanmin(result_df[['home_win_odds', 'draw_odds', 'away_win_odds']].fillna(0.0).to_numpy(), axis=1)

# Assegno ai valori encoded dei valori più comprensibili per vittoria pareggio sconfitta
conditions = [
(model_3_compare['preds'] == 2),  # Condizione per Home Win
(model_3_compare['preds'] == 1),  # Condizione per Away Win
(model_3_compare['preds'] == 0)   # Condizione per Draw
]
conditions_bookmakers = [
(model_3_compare['bookmakers_pred'] == 0),  # Condizione per Home Win
(model_3_compare['bookmakers_pred'] == 2),  # Condizione per Away Win
(model_3_compare['bookmakers_pred'] == 1)   # Condizione per Draw
]

# Valori corrispondenti alle condizioni
values = ['W', 'L', 'D']

# Creazione della nuova colonna 'result' e 'points
model_3_compare['preds'] = np.select(conditions, values)
model_3_compare['bookmakers_pred'] = np.select(conditions_bookmakers, values)
model_3_compare['is_correct'] = model_3_compare['preds'] == model_3_compare['result']
model_3_compare['money_won'] = model_3_compare['pred_prob']*model_3_compare['bookmakers_prob']

In [ ]:
# Visualizziamo i nostri riusltati corretti e quelli sbagliati della bookmakers
test = model_3_compare[(model_3_compare['is_correct']) & (model_3_compare['result']!=model_3_compare['bookmakers_pred'])].sort_values(['money_won'], ascending=False).head(100)
print(f'il nostro modello ha performato meglio in {len(test)} su {len(model_3_compare)}') 
test.head()

In [ ]:
# Visualizziamo i nostri riusltati sbagliati e quelli corretti della bookmakers
test = model_3_compare[(~model_3_compare['is_correct']) & (model_3_compare['result']==model_3_compare['bookmakers_pred'])].sort_values(['money_won'], ascending=False).head(100)
print(f'il nostro modello ha performato peggio in {len(test)} su {len(model_3_compare)}') 
test.head()

In [ ]:
# vediamo in quanti casi avremmo vinto soldi 
test_1 = model_3_compare[(model_3_compare['is_correct']) & (model_3_compare['money_won']>1)].sort_values(['money_won'], ascending=False)
correct_results = model_3_compare[(model_3_compare['is_correct'])]
print(f'il nostro modello ha performato meglio in {len(test_1)} su {len(model_3_compare)} totali e {len(correct_results)} casi in cui la predizione è corretta ') 
test_1.head(20)

In [ ]:
# vediamo in quanti casi avremmo perso soldi 
test_2 = model_3_compare[(model_3_compare['money_won']>1)]
wrong_results = model_3_compare[(~model_3_compare['is_correct'])].sort_values(['money_won'], ascending=False)
print(f'il nostro modello ha performato peggio in {len(test_2)} su {len(model_3_compare)} totali e {len(wrong_results)} casi in cui la predizione è sbagliata ') 
test_2.head(20)

In [ ]:
# vediamo quanti soldi avrei vinto e quanti perso
test_1['bookmakers_prob'].sum()-len(test_2)

### Visualizziamo le nostre predizioni peggiori nel dataframe di test

In [ ]:
model_3_compare[(~model_3_compare['is_correct'])].sort_values('pred_prob',ascending=False).head(15)

il nostro modello ha problemi a predire i pareggi

In [ ]:
len(model_3_compare[(model_3_compare['result']=='D')]), len(model_3_compare[(model_3_compare['result']=='D')&(model_3_compare['is_correct'])])
# non ne ha predetto correttamente neanche uno 

In [ ]:
model_3_compare[(model_3_compare['result']=='D')]

# Test

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

giorni_cumulativi = 2
# calcolo il dataframe con i giorni cumulativi 
df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)

#scelgo le colonne che voglio usare
numero_colonne = 'less'
print(f'numero colonne: {numero_colonne}')    
(X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded, 
               X_train_df, X_valid_df, X_test_df, Train_labels, Valid_labels, Test_labels) = preprocess_columns(
                                                                                        df_Serie_A_days,numero_colonne,giorni_cumulativi, oversample=False)
input_shape = X_train_norm.shape[1]
Dataset_train_norm = tf.data.Dataset.from_tensor_slices((X_train_norm))
Train_labels_encoded = tf.data.Dataset.from_tensor_slices(Train_labels_encoded) # make labels
Dataset_train_norm = tf.data.Dataset.zip((Dataset_train_norm, Train_labels_encoded))

Dataset_valid_norm = tf.data.Dataset.from_tensor_slices((X_valid_norm))
Valid_labels_encoded = tf.data.Dataset.from_tensor_slices(Valid_labels_encoded) # make labels
Dataset_valid_norm = tf.data.Dataset.zip((Dataset_valid_norm, Valid_labels_encoded))

Dataset_test_norm = tf.data.Dataset.from_tensor_slices((X_test_norm))
Test_labels_encoded = tf.data.Dataset.from_tensor_slices(Test_labels_encoded) # make labels
Dataset_test_norm = tf.data.Dataset.zip((Dataset_test_norm, Test_labels_encoded))

Dataset_train_norm = Dataset_train_norm.batch(32).prefetch(tf.data.AUTOTUNE) #Autotune è per dirgli di prefetchare tanti dati quanti può
Dataset_valid_norm = Dataset_valid_norm.batch(32).prefetch(tf.data.AUTOTUNE)
Dataset_test_norm = Dataset_test_norm.batch(32).prefetch(tf.data.AUTOTUNE)

creo e compilo

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

#creo e compilo il modello denso
inputs = layers.Input(shape=(input_shape,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dropout(0.2)(x)  # Aggiunto il layer di dropout per ridurre overfitting
x = layers.Dense(8, activation='relu')(x)
x = layers.Dropout(0.1)(x) 
x = layers.Dense(8, activation='relu')(x)
x = layers.Dropout(0.1)(x) 
outputs = layers.Dense(3, activation = 'softmax')(x)
dense_model = tf.keras.Model(inputs, outputs, name='model_dense')

# compilo il modello 
dense_model.compile(
    loss='sparse_categorical_crossentropy',
    metrics = 'accuracy',
    optimizer = tf.keras.optimizers.SGD()
)

fitto

In [ ]:
#set the seed
tf.random.set_seed(42)
np.random.seed(42)
random.seed(12)

# fitto il modello 
history_test = dense_model.fit(
                x = Dataset_train_norm,
                epochs = 80,
                verbose = 1,
                batch_size = 32,
                validation_data = Dataset_valid_norm,
                callbacks = [
                                #lr_scheduler,
                                #create_model_checkpoint(model_name= 'Dense_model_best_1') 
                            ] 
                )